In [ ]:
import os
import sys
from pathlib import Path

sys.path.append("../")æ©

import numpy as np
import pandas as pd

In [ ]:
from choice_learn.data import ChoiceDataset
from choice_learn.data.storage import FeaturesStorage
# path to dataset file is:
filepath = "choice_learn/datasets/data/ModeCanada.csv.gz"
canada_transport_df = pd.read_csv((Path("..") / filepath), index_col=0)

print("Let's look at the dataframe:")
canada_transport_df.head()

In [ ]:
load_modecanada = 1

In [ ]:
"""
from choice_learn.models import ConditionalLogitModel

model = ConditionalLogitModel()
model.add_coefficient(coefficient_name="beta_customer_income",
                      feature_name="customer_income",
                      items_indexes=[0, 1, 2])
model.add_coefficient(coefficient_name="beta_price",
                      feature_name="price",
                      items_indexes=[0, 1, 2])
model.add_shared_coefficient(coefficient_name="beta_travel_time",
                             feature_name="travel_time",
                             items_indexes=[0, 1, 2])
model.add_shared_coefficient(coefficient_name="conditional_intercept",
                             feature_name="is_individual",
                             items_indexes=[0, 1, 2])

model.compile()
model.fit(dataset)
model.evaluate()
"""

In [ ]:
transport_df = canada_transport_df.copy()
items = ["air", "bus", "car", "train"]

transport_df["oh0"] = transport_df.apply(lambda row: 1. if row.alt == items[0] else 0., axis=1)
transport_df["oh1"] = transport_df.apply(lambda row: 1. if row.alt == items[1] else 0., axis=1)
transport_df["oh2"] = transport_df.apply(lambda row: 1. if row.alt == items[2] else 0., axis=1)
transport_df["oh3"] = transport_df.apply(lambda row: 1. if row.alt == items[3] else 0., axis=1)

# Just some typing
transport_df.income = transport_df.income.astype("float32")

In [ ]:
transport_df

In [ ]:
cf = np.array([[1], [2], [3], [4]])
cif = np.array([[[1], [2], [3], [4]], [[1], [2], [3], [4]], [[1], [2], [3], [4]], [[1], [2], [3], [4]]])
av = np.array([[1, 1, 1, 1], [1, 0, 0, 1], [0, 1, 1, 1], [1, 1, 0, 0]])

In [ ]:
dataset = ChoiceDataset(fixed_items_features = np.eye(4),
                        contexts_features=cf,
                        contexts_items_features=cif,
                        contexts_items_availabilities=av,
                        choices=[0, 3, 2, 1],
                        fixed_items_features_names=["air", "bus", "car", "train"],
                        contexts_features_names=["income"],
                        contexts_items_features_names=["zut"],
                        )

In [ ]:
fif_df = pd.DataFrame(np.eye(4), columns=["air", "bus", "car", "train"], index=["air", "bus", "car", "train"])

cif_df = {
    "item_id": ["air", "bus", "car", "train", "air", "bus", "car", "train", "air", "bus", "car", "train", "air", "bus", "car", "train"],
    "session_id": [0, 0, 0, 0, 1, 1, 1, 1, 2, 2, 2, 2, 3, 3, 3, 3],
    "zut": [1, 2, 3, 4, 1, 2, 3, 4, 1, 2, 3, 4, 1, 2, 3, 4],
}
cif_df = pd.DataFrame(cif_df)

cf_df = pd.DataFrame(cf, columns=["income"])

dataset = ChoiceDataset(fixed_items_features=fif_df,
                        contexts_features=cf_df,
                        contexts_items_features=cif_df,
                        contexts_items_availabilities=av,
                        choices=[0, 3, 2, 1],
                        )

In [ ]:
dataset.fixed_items_features_names

In [ ]:
dataset.contexts_items_features_names

In [ ]:
dataset.contexts_features_names

In [ ]:
dataset.summary()

In [ ]:
storage = FeaturesStorage(values={"a": [1, 1], "b": [2, 2]}, name="test")

In [ ]:
test_df = transport_df[["income"]]
test_df["test"] = "a"

In [ ]:
dataset = ChoiceDataset(fixed_items_features=np.eye(4),
contexts_features=test_df,
choices=np.ones((len(test_df), 1)),
features_by_ids=[storage])

In [ ]:
dataset.contexts_features[0][[0, 1]]

In [ ]:
dataset.get_choices_batch([0, 1]), dataset.batch[[0, 1]]

In [ ]:
batch = (dataset.contexts_features[0][:16], )
batch

In [ ]:
transport_df

In [ ]:
dataset = ChoiceDataset.from_single_df(
    df=transport_df,
    fixed_items_features_columns=["oh0", "oh1", "oh2", "oh3"],
    contexts_features_columns=["income"],
    contexts_items_features_columns=["dist", "cost", "freq", "ivt", "ovt"],
    items_id_column="alt",
    contexts_id_column="case",
    choices_column="choice",
    choice_mode="one_zero",
    )

In [ ]:
len(dataset)

In [ ]:
dataset[[0, 1, 2, 3, 4]]

In [ ]:
dataset.get_choices_batch([0, 1])

In [ ]:
dataset.batch[[0, 1]]

In [ ]:
for a, b in zip(dataset.batch[[0, 1]], dataset.get_choices_batch([0, 1])):
    for _a, _b in zip(a, b):
        if not ((_a == _b).all()):
            print(_a, _b)
            print(_a==_b)

In [ ]:
type(dataset.batch[[0, 1]][2][0][0][2][1]),  type(dataset.get_choices_batch([0, 1])[2][0][0][2][1])